![JPL Logo](https://www.jpl.nasa.gov/assets/images/logo_nasa_trio_black@2x.png)  
# Toward an Engineering Notebook for Systems Modeling

## *13 November 2019*
## *Ed Seidewitz, Chief Technology Officer*

<img src="http://www.modeldriven.com/images/MDS%20Logo%20Verticle.gif" height="400" width="400" align="left">


# SysML v2

# Systems Modeling Language™ (SysML<sup>®</sup>)
Supports the specification, analysis, design, and verification and validation of complex systems that may include hardware, software, information, processes, personnel, and facilities

### SysML v1.0 adopted in 2006
Adopted by the Object Management Group (OMG) as a standard profile of UML v2.1

### Current version is SysML v1.6 (beta)
v1.7 is expected to be the last for SysML v1

### Two SysML v2 RFPs have been issued
Language in December 2017 (https://www.omg.org/cgi-bin/doc?ad/17-12-02)

API and Services in June 2018 (https://www.omg.org/cgi-bin/doc?ad/18-06-03)

## SysML v2 Submission Team (SST)
A broad team drawn from end user, vendor, academic and government organizations

Currently 120+ members from 60+ organizations

### Developing submissions to both RFPs

### Driven by RFP requirements and user needs

### Current Schedule
Initial submission in June 2020

Revised submission in June 2021

Full commercial/production implementations expected by 2022

## SysML v2 Pilot Implementation
The SST is committed to fully prototyping SysML v2 in a pilot implementation before submitting the complete language proposal.

### Open source, with support from some commercial tooling

### Developed incrementally
Monthly internal releasees

Quarterly public releases
* First release available at http://openmbee.org/sysml-v2-release/2019-09

## SysML v2 Status

### Currently focusing on textual notation, with limited graphical visualization
SysML v2 will have a complete textual as well as graphical notation

Textual notation can be implemented more quickly and incrementally than graphical notation

### First pubic release available at http://openmbee.org/sysml-v2-release/2019-09
Initial capabilities for integrated structural and behavioral modeling
* Block definition and decomposition
* Ports and connectors
* Activities and actions
* "Usage-focused" moodeling

# Jupyter

## Project Jupyter

"Project Jupyter exists to develop open-source software, open-standards, and services for interactive computing" (http://jupyter.org)

### Interactive programming
Originally based on the Python programming language

Now supports "dozens of programming languages"

### Jupyter Notebook
"The Jupyter Notebook is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and narrative text." 

## Jupyter at JPL

### Started with grassroots use

### More recently supported by CAE
JPL JupyterHub installation at https://cae-jupyter.jpl.nasa.gov/ (internal access)

### Increasingly integrated with other products
JPL propopsed integrating the SysML v2 into Jupyter

## Jupyter and SysML v2
The SysML v2 textual notation is integrated as another Jupyter "language kernel"

### Goals

*Near term*
* A way to quickly create SysML v2 models
* A "no install" platform to track the latest incremental releases of the language 
  * via the JupyterHub deployment available to SST members at http://jupyter.openmbee.org

*Long term*
* A model-based engineering analysis environment
* A means to interchange and integrate system engineering models

# Mass Rollup Demo

## Massed Things

To create a simple mass rollup example, we first define the general concept of a `MassedThing` with two mass properties:
* `mass` - The mass of the thing itself, not including the mass of any composite parts it may have.
* `totalMass` - The total mass of the thing, including the mass of its composite parts.

In [12]:
block MassedThing {
    value mass :> ISQ::mass; 
    value totalMass :> ISQ::mass;
}

Block MassedThing (5a84b767)


## Simple and Composite Things

Next we define two kinds of usages of `MassedThing`:
* `simpleThing`, which has no composite parts, so its `totalMass` is just the same as its `mass`.
* `compositeThing`, which has composite parts (`subcomponents`), so its `totalMass` is the some of its `mass` plus the `totalMass` of all its composite parts.

In [13]:
part simpleThing : MassedThing {
    value totalMass redefines MassedThing::totalMass = mass;
}

part compositeThing : MassedThing {
    import RealFunctions::sum; 
    
    part subcomponents: MassedThing[*];

    value totalMass redefines MassedThing::totalMass =
        mass + sum(subcomponents.totalMass); 
}

PartProperty simpleThing (1499759c)
PartProperty compositeThing (2a95f03e)


## Spacecraft Components and Systems

Then we can apply the general model to specify that a `SpacecraftComponent` is a `MassedThing` and a `spacecraftSystem` is a composite `SpacecraftComponent` (that is, both a `SpacecraftComponent` _and_ a `compositeThing`) whose `subcomponents` are its `subSystems`.

In [14]:
block SpacecraftComponent :> MassedThing {
    value identifier: ScalarValues::String;
}

part spacecraftSystem: SpacecraftComponent :> compositeThing {
    part subsystems[*] :> spacecraftSystem redefines subcomponents;    
}

Block SpacecraftComponent (266a6235)
PartProperty spacecraftSystem (7ae2ee68)


## Spacecraft Decomposition

A `spacecraft` is just a top-level `spacecraftSystem` whose subsytems are the main spacecraft systems.

In [15]:
block PropulsionSystem :> SpacecraftComponent { 
    // ...
}

block PowerSystem :> SpacecraftComponent { 
    // ...
}

part spacecraft :> spacecraftSystem {
    part propulsion: PropulsionSystem subsets subsystems;    
    part power: PowerSystem subsets subsystems;
    // etc...
}

Block PropulsionSystem (6afc43e0)
Block PowerSystem (5e1213fe)
PartProperty spacecraft (521d9522)


## Specific Spacecraft Configuration

A configuration for a specific spacecraft gives the masses for each of the subsystems -- and can even add additional subsystems as appropriate.

In [19]:
part spacecraft_1 :> spacecraft {
    import SI::*;
    
    redefines mass = 1000@[kg];
    
    part redefines propulsion {
        redefines mass = 100@[kg];
    }
    
    part redefines power {
        redefines mass = 50@[kg];
        part battery subsets subsystems {
            redefines mass = 10@[kg];
        }
    }
}

PartProperty spacecraft_1 (23364f5e)


If the `spacecraft_1::totalMass` was properly evaluated above, the result should be `1160.0@[kg]`.

# Vehicle Model Example

In [ ]:
package VehicleModel_1{
    package ModelLibrary{
        package PartDefinitions{
            block Vehicle {
                value mass :> ISQ::mass;
            }
            block Engine;
            block Cylinder;
            block Transmission;
            block Driveshaft;
            block AxleAssembly;
            block Axle;
            block FrontAxle:>Axle{
                value steeringAngle:ScalarValues::Real;
            }
            block HalfAxle;
            block Differential;
            block Wheel;
        }
        package PortDefinitions{
            port def FuelCmdPort;
            port def DrivePwrPort;
            port def ClutchPort;
            port def ShaftPort_a;
            port def ShaftPort_b;
            port def ShaftPort_c;
            port def ShaftPort_d;
            port def DiffPort;
            port def AxlePort;
            port def AxleToWheelPort;
            port def WheelToAxlePort;
            port def WheelToRoadPort;
            port def VehicleToRoadPort;
        }
        package FlowDefinitions{
            // This defines Torque as an alias for ISQ::Torque.
            value type Torque is ISQ::TorqueValue;
            value type FuelCmd;
        }
        package InterfaceDefinitions{
            import PortDefinitions::*;
            interface def EngineToTransmissionInterface{
                end p1:DrivePwrPort;
                end p2:ClutchPort;
            }
        }
        package ActionDefinitions{
            import FlowDefinitions::*;
            activity ProvidePower (
                in fuelCmd:FuelCmd,
                out wheelToRoadTorque:Torque[2]
            );
            activity GenerateTorque (
                in fuelCmd:FuelCmd,
                out engineTorque:Torque
            );
            activity AmplifyTorque (
                in engineTorque:Torque,
                out transmissionTorque:Torque
            );
            activity TransferTorque (
                in transmissionTorque:Torque,
                out driveshaftTorque:Torque
            );
            activity DistributeTorque (
                in driveshaftTorque:Torque,
                out wheelToRoadTorque:Torque[2]
            );
        }
        package ValueDefinitions{
            import ScalarValues::*;
        }
    }
    package VehicleConfigurations{
        import ModelLibrary::*;
        import ModelLibrary::PartDefinitions::*;
        import ModelLibrary::PortDefinitions::*;
        import ModelLibrary::FlowDefinitions::*;
        import ModelLibrary::InterfaceDefinitions::*;
        import ModelLibrary::ActionDefinitions::*;
        import ModelLibrary::ValueDefinitions::*;
        
        package VehicleConfiguration_a{
            package VehiclePartsTree{
                part vehicle_a:Vehicle{
                    value mass redefines mass=1750;
                    part frontAxleAssembly:AxleAssembly{
                        part frontAxle:Axle;
                        part frontWheels:Wheel[2];
                    }
                    part rearAxleAssembly:AxleAssembly{
                        part rearAxle:Axle;
                        part rearWheels:Wheel[2];
                    }
                }
            }
            package VehicleActionTree{
                
            }
        }
        package VehicleConfiguration_b{
            import VehicleConfiguration_a::*;
            import VehicleConfiguration_a::VehiclePartsTree::*;
            package VehiclePartsTree{
                part vehicle_b :> vehicle_a{
                     value mass redefines vehicle_a::mass=2000;
                    /*adding ports to parts. will add ports to blocks later and redefine here*/
                    port fuelCmdPort:FuelCmdPort;
                    port vehicleToRoadPort:VehicleToRoadPort{
                        port wheelToRoadPort1:WheelToRoadPort;
                        port wheelToRoadPort2:WheelToRoadPort;
                    }
                    perform VehicleActionTree::providePower;
                    part frontAxleAssembly redefines vehicle_a::frontAxleAssembly{
                        port shaftPort_d:ShaftPort_d;
                        part frontAxle:FrontAxle redefines vehicle_a::frontAxleAssembly::frontAxle;
                        part frontWheel1:>frontWheels=frontWheels[1];
                        part frontWheel2:>frontWheels=frontWheels[2];
                    }
                    part rearAxleAssembly redefines vehicle_a::rearAxleAssembly{
                        port shaftPort_d:ShaftPort_d;
                        perform VehicleActionTree::providePower::distributeTorque;
                        part differential:Differential{
                            port shaftPort_d:ShaftPort_d;
                            port leftDiffPort:DiffPort;
                            port rightDiffPort:DiffPort;
                        }
                        part rearAxle redefines vehicle_a::rearAxleAssembly::rearAxle{
                            part leftHalfAxle:HalfAxle{
                                port leftAxleToDiffPort:AxlePort;
                                port leftAxleToWheelPort:AxlePort;
                            }
                            part rightHalfAxle:HalfAxle{
                                port rightAxleToDiffPort:AxlePort;
                                port rightAxleToWheelPort:AxlePort;
                            }
                        }
                        part rearWheel1:>rearWheels=rearWheels[1]{
                            port wheelToAxlePort:WheelToAxlePort;
                            port wheelToRoadPort:WheelToRoadPort;
                        }
                        part rearWheel2:>rearWheels=rearWheels[2]{
                            port wheelToAxlePort:WheelToAxlePort;
                            port wheelToRoadPort:WheelToRoadPort;
                        }
                        bind shaftPort_d=differential::shaftPort_d;
                        connect differential::leftDiffPort to rearAxle::leftHalfAxle::leftAxleToDiffPort;
                        connect differential::rightDiffPort to rearAxle::rightHalfAxle::rightAxleToDiffPort;
                        connect rearAxle::leftHalfAxle::leftAxleToDiffPort to rearWheel1::wheelToAxlePort;
                        connect rearAxle::rightHalfAxle::rightAxleToDiffPort to rearWheel2::wheelToAxlePort;
                    }
                    part engine:Engine{
                        port fuelCmdPort:FuelCmdPort;
                        port drivePwrPort:DrivePwrPort{
                            out engineTorque:Torque;
                        }
                        part cylinders:Cylinder[4];
                        perform VehicleActionTree::providePower::generateTorque;
                    }
                    part transmission:Transmission{
                        port clutchPort:ClutchPort{
                            in engineTorque:Torque;
                        }
                        port shaftPort_a:ShaftPort_a;
                        perform VehicleActionTree::providePower::amplifyTorque;
                    }
                    part driveshaft:Driveshaft{
                        port shaftPort_b:ShaftPort_b;
                        port shaftPort_c:ShaftPort_c;
                        perform VehicleActionTree::providePower::transferTorque;
                    }
                    bind vehicle_b::fuelCmdPort=vehicle_b::engine::fuelCmdPort;
                    connect engine::drivePwrPort to transmission::clutchPort; 
                    interface engineToTransmissionInterface:EngineToTransmissionInterface
                        connect p1=>engine::drivePwrPort to p2=>transmission::clutchPort{
                             perform VehicleActionTree::providePower::generateToAmplify;
                        }
                    connect transmission::shaftPort_a to driveshaft::shaftPort_b; 
                    connect driveshaft::shaftPort_c to rearAxleAssembly::shaftPort_d;
                    bind rearAxleAssembly::rearWheel1::wheelToRoadPort=vehicle_b::vehicleToRoadPort::wheelToRoadPort1;
                    bind rearAxleAssembly::rearWheel2::wheelToRoadPort=vehicle_b::vehicleToRoadPort::wheelToRoadPort2;
                }
            }
            package VehicleActionTree{
                action providePower:ProvidePower{
                    // No successions (control flows) between these actions, because the
                    // flows between them are continuous streams.
                    action generateTorque:GenerateTorque (
                        in = providePower::fuelCmd,
                        out engineTorque);
                    action amplifyTorque:AmplifyTorque (
                        in engineTorque,
                        out transmissionTorque
                    );
                    stream generateToAmplify from generateTorque::engineTorque to amplifyTorque::engineTorque;
                    action transferTorque:TransferTorque (
                        in transmissionTorque stream from transferTorque::transmissionTorque
                    );
                    action distributeTorque:DistributeTorque;
                }
            }
        }
    }
}

## "Magic" Commands

## Showing Elements

The `%show <name>` "magic" command shows the parsed syntax tree for the named element.

```sysml
part spacecraft :> spacecraftSystem {
    part propulsion: PropulsionSystem subsets subsystems;    
    part power: PowerSystem subsets subsystems;
}
```

In [20]:
%show spacecraft

PartProperty spacecraft (521d9522)
  [Subsetting] PartProperty spacecraftSystem (7ae2ee68)
  [FeatureMembership] PartProperty propulsion (42d2d928)
    [FeatureTyping] Block PropulsionSystem (6afc43e0)
    [Subsetting] PartProperty subsystems (58872da2)
  [FeatureMembership] PartProperty power (c11bf6f)
    [FeatureTyping] Block PowerSystem (5e1213fe)
    [Subsetting] PartProperty subsystems (58872da2)


## Publishing Models

The `%publish <name>` command "publishes" the model graph rooted in the named element to a prototype repository via the proposed standard API. 

* The API endpoint currently used is for a single, publicly available instance of the repository.
* All elements directly *and* indirectly accessible from the specified root element are saved -- even if they come from library models. (This means that even small user models can result in a lot being saved.)
* Each publish of a model creates a new copy in the repository.

## Visualizing Models

Tom Saywer Software is developing graphical visualization for SysML v2. A prototype tool is currently available at http://solutions.tomsawyer.com:8080/SysML2 to visualize models published to the repository.

# Moving Forward



## Next public release: 2019-12
Available early January 2020

### State machines
### Individual and snapshot modeling
### Constraints (?)

## Additional Jupyter notebook capabiities

### Import of published models
### Incremental update of published models
### In-notebook visualization
### Integration with analysis tools